# Text Feature Extractor (Sentence Level)
_Click 'Run All' to extract text features from a sentence input._
***

## Traditional Text Features

#### Library Imports & Set-up

In [78]:
#import libraries for traditional text features
import os
from os import path

import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.probability import FreqDist
import string
import re
import syllables
import csv
import pandas as pd
import numpy as np
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

import stopwordsiso
from stopwordsiso import stopwords

import matplotlib.pyplot as plt
%matplotlib inline

nltk.download('punkt')

#import libraries for lexical text features
import os
import nltk
from nltk import *
from nltk.tag.stanford import StanfordPOSTagger
from nltk.tokenize import word_tokenize
import math


# POS-Tagger SET UP
# input local path to java.exe
java_path = "C:/Program Files/Java/jre1.8.0_341/bin/java.exe" 
os.environ["JAVAHOME"] = java_path

#path to POS tagger jar
jar_path = 'C:/Users/Alia/Documents/school/THESIS/Readability-Level-Identifier'
os.chdir(jar_path)
jar =  jar_path + "/stanford-postagger.jar"

# path to POS tagger model
model_path = jar_path +"/POSTagger/"
model = model_path + "filipino-left5words-owlqn2-distsim-pref6-inf2.tagger"

pos_tagger = StanfordPOSTagger(model, jar, encoding = "utf-8")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Alia\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


#### Word Counter

In [79]:
def word_count(text):
    words = text.split()
    return len(words)

#### Average Word Length

In [80]:
def avg_word_length(text):
    words = text.split()
    total_word_length = sum(len(word) for word in words)
    avg = total_word_length / len(words)

    return avg

# print("Average Word Length: ", avg_word_length, "letters")

#### Total Syllables

In [81]:
# https://github.com/itudidyay/Tagalog-Word-Syllabization-Python
# https://pypi.org/project/syllables/

vowels = 'aeiou'
consonants = 'bcdfghjklmnpqrstvwxyz'

def count_syllables(text):

    total_syllables = 0
    monosyl_count = 0
    polysyl_count = 0
    
    tokens = word_tokenize(text)

    for token in tokens:
        syllable_count = 0
        for char in token:
            if char.lower() in vowels:
                total_syllables += 1
                syllable_count += 1
        
        # edge cases
        if token == 'ng' or token == 'mga': # edge case ng, mga
            total_syllables += 1
            syllable_count += 1
        
        elif (('io') in token): # edge case -io in names/surnames
            total_syllables -= 1
            syllable_count -= 1
            
        if syllable_count == 1:
            monosyl_count += 1
        elif syllable_count > 1:
            polysyl_count += 1

    return total_syllables, monosyl_count, polysyl_count

# def main():
#     total_syllables = count_syllables(clean_input)

#     print(f"Total syllables in the text file: {total_syllables}")
#     print(f"Number of monosyllabic words: {monosyl_count}")
#     print(f"Number of polysyllabic words: {polysyl_count}")

# if __name__ == "__main__":
#     main()

***
***
## Lexical Text Features
#### Input Tokenization & POS Tagging

In [82]:
# #tokenize text input
# words = nltk.word_tokenize('nagsimula ang lahat sa masukal na bakuran ni aling salvacion')
# print(words)

# #tag tokenized words
# tagged_words = pos_tagger.tag(words)

#### Noun-Token Ratio

In [83]:
def ntr(words, tagged):

    # NOUN COUNT
    noun_count = 0
    for word, tag in tagged:
        tag = tag.split('|')[-1] #removes word before |
        if tag.startswith('NN'):
            noun_count += 1
        
    # print("Number of nouns: ", noun_count)

    # NOUN TOKEN RATIO
    # = noun_count/total_token_count
    total_token_count = len(words)
    noun_token_ratio = noun_count/total_token_count

    return noun_token_ratio

# print("Total number of tokens: ", total_token_count)
# print("Noun-Token Ratio: ", noun_token_ratio)

#### Verb-Token Ratio

In [84]:
def vtr(words, tagged):
    # VERB COUNT
    verb_count = 0
    for word, tag in tagged:
        tag = tag.split('|')[-1] #removes word before |
        if tag.startswith('VB'):
            verb_count += 1
            
    # print("Number of verbs: ", verb_count)

    # VERB TOKEN RATIO
    # = verb_count/total_token_count
    total_token_count = len(words)
    verb_token_ratio = verb_count/total_token_count

    return verb_token_ratio

# print("Total number of tokens: ", total_token_count)
# print("Noun-Token Ratio: ", verb_token_ratio)

#### Type-Token Ratio

In [85]:
def ttr(words, tagged):
# count unique lexical categories
    unique_categories = set()
    for _, tag in tagged:
        tag = tag.split('|')[-1] #removes word before |
        if len(tag) >= 2:  # make sure the tag is not empty
            category = tag[:2]  # extract the first two letters
            unique_categories.add(category)

    # print("Unique Categories:", unique_categories)

    #NUMBER OF UNIQUE CATEGORIES
    num_categories = len(unique_categories)
    # print("Number of Unique Categories:", num_categories)

    # TOTAL NUM OF TOKENS
    total_token_count = len(words)

    # TYPE TOKEN RATIO
    ttr = num_categories/total_token_count
    # print("Type-Token Ratio: ", ttr)

    #ROOT TTR
    root_ttr = num_categories/math.sqrt(total_token_count)
    # print("Root Type-Token Ratio: ", root_ttr)

    #CORR TTR
    corr_ttr = num_categories/math.sqrt(2*total_token_count)
    # print("Corrected Type-Token Ratio: ", corr_ttr)

    #BILOGARITHMIC TTR
    denominator = math.log(total_token_count)

    if denominator == 0:
        log_ttr = 0
    else:
        log_ttr = math.log(num_categories)/math.log(total_token_count)
    # print("Bilogarithmic Type-Token Ratio: ", log_ttr)

    return ttr, root_ttr, corr_ttr, log_ttr

#### Lexical Density

In [86]:

def lexical_density(words, tagged):

    # NUMBER OF LEXICAL WORDS
    # count number of nouns, verbs, adjectives, and adverbs
    num_lexwords = 0
    for word, tag in tagged:
        tag = tag.split('|')[-1] #removes word before |
        if tag.startswith('NN') or tag.startswith('VB') or tag.startswith('JJ') or tag.startswith('RB'):
            num_lexwords += 1
            
    # print("Number of lexical words: ", num_lexwords)

    # LEXICAL DENSITY
    # = lex_density/total_token_count
    total_token_count = len(words)
    lex_density = num_lexwords/total_token_count

    return lex_density

# print("Total number of tokens: ", total_token_count)
# print("Lexical Density: ", lex_density)

#### Foreign Word-Token Ratio

In [87]:
def fwtr(words, tagged):
    # FOREIGN WORD COUNT
    fw_count = 0
    for word, tag in tagged:
        tag = tag.split('|')[-1] #removes word before |
        if tag.startswith('FW'):
            fw_count += 1
            
    # print("Number of foreign words: ", fw_count)

    # FOREIGN WORD - TOKEN RATIO
    # = fw_count/total_token_count
    total_token_count = len(words)
    fw_token_ratio = fw_count/total_token_count

    return fw_token_ratio

# print("Total number of tokens: ", total_token_count)
# print("Foreign Word-Token Ratio: ", fw_token_ratio)

#### Sentence-Level Input

In [88]:
path = 'C:/Users/Alia/Documents/school/THESIS/Readability-Level-Identifier/archive/token-sentences'
os.chdir(path)

csv_header = ['Word Count', 'AVG Word Length', 'Total Syllables', 'MONOSYLL', 'POLYSYLL', 'NTR', 'VTR', 'TTR', 'Root TTR', 'Corrected TTR', 'BiLog TTR', 'LD', 'FWTR', 'Age Classification']
data = []

def read_text_file(file_path):
    path, file_name = os.path.split(file_path)
    suffix = '_sentenceTokens.txt'
    if file_name.lower().endswith(suffix.lower()):
        file_name = file_name[: -len(suffix)]   # e.g. Tahan na Tahanan

    with open(file_path, 'r') as file:
        print('-------------------------------------------------')
        print(file_name)
        print('-------------------------------------------------')
        
        sentence = 0
        for line in file:

            # added the age at Line 0 of each sentence token script          
            if sentence == 0:
                age = line.strip()  # output: X-Y
                sentence += 1
                continue

            # csv_data = [file_name]
            csv_data = [] # excludes title column

            # print('Sentence ', sentence)

            # TRAD
            # print('WORD COUNT: ', word_count(line))
            csv_data.append(word_count(line))

            # print('AVG WORD LENGTH: ', avg_word_length(line))
            csv_data.append(avg_word_length(line))

            # print('TOTAL SYLLABLES: ', count_syllables(line)[0])
            # print('MONOSYLLABIC: ', count_syllables(line)[1])
            # print('POLYSYLLABIC: ', count_syllables(line)[2])
            csv_data.append(count_syllables(line)[0])
            csv_data.append(count_syllables(line)[1])
            csv_data.append(count_syllables(line)[2])


            # LEX
            wordsss = nltk.word_tokenize(line)
            tagged_words = pos_tagger.tag(wordsss)
            # print('NTR: ', ntr(wordsss, tagged_words))
            csv_data.append(ntr(wordsss, tagged_words))

            # print('VTR: ', vtr(wordsss, tagged_words))
            csv_data.append(vtr(wordsss, tagged_words))

            # print('TTR: ', ttr(wordsss, tagged_words)[0])
            # print('Root-TTR: ', ttr(wordsss, tagged_words)[1])
            # print('Corrected-TTR: ', ttr(wordsss, tagged_words)[2])
            # print('Bilogarithmic-TTR: ', ttr(wordsss, tagged_words)[3])
            csv_data.append(ttr(wordsss, tagged_words)[0])
            csv_data.append(ttr(wordsss, tagged_words)[1])
            csv_data.append(ttr(wordsss, tagged_words)[2])
            csv_data.append(ttr(wordsss, tagged_words)[3])

            # print('Lexical Density: ', lexical_density(wordsss, tagged_words))
            csv_data.append(lexical_density(wordsss, tagged_words))

            # print('FWTR: ', fwtr(wordsss, tagged_words))
            csv_data.append(fwtr(wordsss, tagged_words))

            csv_data.append(age)
            data.append(csv_data)
            print(data)

            # hanggang dito lang kaya ng makina ko D:
            if sentence == 10:
                break
            else:
                sentence += 1

    return '0'

for file in os.listdir(path):
    if file.endswith('.txt'):
        file_path = f'{path}/{file}'
        read_text_file(file_path)

main = 'C:/Users/Alia/Documents/school/THESIS/Readability-Level-Identifier'
os.chdir(main)

df = pd.DataFrame(data, columns = csv_header)
df.to_csv('sentence.csv')

-------------------------------------------------
Ang Aklatang Pusa
-------------------------------------------------


[[10, 5.1, 21, 4, 6, 0.2, 0.1, 0.6, 1.8973665961010275, 1.3416407864998738, 0.7781512503836435, 0.4, 0.0, '8-12']]
[[10, 5.1, 21, 4, 6, 0.2, 0.1, 0.6, 1.8973665961010275, 1.3416407864998738, 0.7781512503836435, 0.4, 0.0, '8-12'], [15, 5.066666666666666, 34, 4, 11, 0.26666666666666666, 0.06666666666666667, 0.4, 1.5491933384829668, 1.0954451150103321, 0.6616418958920284, 0.4, 0.0, '8-12']]
[[10, 5.1, 21, 4, 6, 0.2, 0.1, 0.6, 1.8973665961010275, 1.3416407864998738, 0.7781512503836435, 0.4, 0.0, '8-12'], [15, 5.066666666666666, 34, 4, 11, 0.26666666666666666, 0.06666666666666667, 0.4, 1.5491933384829668, 1.0954451150103321, 0.6616418958920284, 0.4, 0.0, '8-12'], [6, 4.333333333333333, 11, 2, 4, 0.3333333333333333, 0.0, 0.6666666666666666, 1.6329931618554523, 1.1547005383792517, 0.7737056144690831, 0.5, 0.0, '8-12']]
[[10, 5.1, 21, 4, 6, 0.2, 0.1, 0.6, 1.8973665961010275, 1.3416407864998738, 0.7781512503836435, 0.4, 0.0, '8-12'], [15, 5.066666666666666, 34, 4, 11, 0.26666666666666666, 0.066